<a href="https://colab.research.google.com/github/ygGao1120/Onclusive-data-challenge/blob/main/Final_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('drive/MyDrive/interview/Onclusive data assignment')
!ls

In [ ]:
pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import torch
#import transformers
import warnings

warnings.filterwarnings('ignore')

from torch import nn
from torch import optim
import transformers as tfs
import math

In [ ]:
df_train = pd.read_excel('cleaned_train.xlsx')
df_valid = pd.read_excel('cleaned_valid.xlsx')
df_test = pd.read_excel('cleaned_test.xlsx')

In [ ]:
class BertClassificationModel(nn.Module):
    def __init__(self):
        super(BertClassificationModel, self).__init__()   
        model_class, tokenizer_class, pretrained_weights = (tfs.BertModel, tfs.BertTokenizer, "monologg/biobert_v1.1_pubmed")         
        self.tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
        self.bert = model_class.from_pretrained(pretrained_weights)
        self.dense = nn.Linear(768, 4)  # a linear layer for classification, 4 means there are 4 types of label in total
        
    def forward(self, batch_sentences):
        batch_tokenized = self.tokenizer.batch_encode_plus(batch_sentences, add_special_tokens=True,
                                max_length=100, pad_to_max_length=True)      #tokenize、add special token、pad
        input_ids = torch.tensor(batch_tokenized['input_ids'])#GPU
        attention_mask = torch.tensor(batch_tokenized['attention_mask'])#GPU
        bert_output = self.bert(input_ids, attention_mask=attention_mask)
        bert_cls_hidden_state = bert_output[0][:,0,:]       #get [CLS]
        linear_output = self.dense(bert_cls_hidden_state)
        return linear_output

In [ ]:
#merge training set and validation set together as the new training set
train = pd.concat([df_train,df_valid])

In [ ]:
train.shape

In [ ]:
#creat a column that has the information from both claim and explanation
train['add'] = train.apply(lambda x: [x.clean_claim] + [x.clean_explanation],axis = 1)
df_test['add'] = df_test.apply(lambda x: [x.clean_claim] + [x.clean_explanation],axis = 1)

In [ ]:
train_inputs = train['add'].values
test_inputs = df_test['add'].values

train_targets = train['label'].values
test_targets = df_test['label'].values

# spilt the data based on batch size
batch_size = 64
batch_count = int(len(train_inputs) / batch_size)
batch_train_inputs, batch_train_targets = [], []
for i in range(batch_count):
    batch_train_inputs.append(train_inputs[i*batch_size : (i+1)*batch_size])
    batch_train_targets.append(train_targets[i*batch_size : (i+1)*batch_size])

In [ ]:
train_targets.shape

In [ ]:
#training
epochs = 1
lr = 4e-5
print_every_batch = 5
bert_classifier_model = BertClassificationModel()
optimizer = torch.optim.Adam(bert_classifier_model.parameters(),lr=lr)#Adam optimizer
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    print_avg_loss = 0
    for i in range(batch_count):
        inputs = batch_train_inputs[i]
        labels = torch.tensor(batch_train_targets[i])#GPU
        optimizer.zero_grad()
        outputs = bert_classifier_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        print_avg_loss += loss.item()
        if i % print_every_batch == (print_every_batch-1):
            print("Batch: %d, Loss: %.4f" % ((i+1), print_avg_loss/print_every_batch))
            print_avg_loss = 0

In [ ]:
test_hit = 0
total = len(test_inputs)
for i in range(total):
    outputs = bert_classifier_model([test_inputs[i]])
    _,predict = torch.max(outputs,1)
    if predict==test_targets[i]:
        test_hit+=1
#print('Test accuracy is %.4f'%(test_hit/len(test_inputs)))

In [ ]:
print('Test accuracy is %.4f'%(test_hit/len(test_inputs)))

In [ ]:
torch.save(bert_classifier_model, r'\Final_model.pkl')